In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

from utilities import save_dataset

In [2]:
WORK_DIRECTORY = "data"
DATA_FILE_NAME = "leukemia_met"
DATA_FILE_NAME_NEW = "leukemia_met_trans"

In [3]:
# create spark session
spark = (
    SparkSession.builder.master("local")
    .appName("Colab")
    .config("spark.driver.memory", "32g")
    .config("spark.executor.cores", "1")
    .config("spark.cores.max", "1")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size", "100g")
    .config("spark.sparkContext.setLogLevel", "ERROR")
    .getOrCreate()
)

spark.conf.set("spark.sql.pivotMaxValues", 500000)

23/02/18 17:20:30 WARN Utils: Your hostname, maria-HP-EliteBook-840-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.1.160 instead (on interface wlp0s20f3)
23/02/18 17:20:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/18 17:20:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# create spark dataframe
df = spark.read.csv(
    os.path.join(WORK_DIRECTORY, DATA_FILE_NAME),
    header=True,
    inferSchema=True,
    sep=";",
)

In [7]:
class DataTranspose:
    """
    Class transform data {index:methylation_site, col:sample_id}
            to data {index:sample_id, cols:methylation_site}.

    Methods:
    1. unpivot_data: method transform 2D dataset to 1D dataset.
    2. pivot_data: method transform 1D dataset to 2D dataset.

    """

    def __init__(self, data):
        self.data = data
        self.columns = data.columns[1:-1]

    def unpivot_data(self):
        cols_expr = ", ".join(["".join(
            ["'", col, "', ", col]) for col in self.columns])
        unpivotExpr = "".join(
            ["stack(", str(len(self.columns)), ", ", cols_expr, ") as (s,v)"]
        )
        self.data = self.data.select("sample_id", expr(unpivotExpr)).where(
            "v is not null"
        )
        self.data = self.data.withColumn("v", self.data.v.cast("double"))

    def pivot_data(self):
        self.data = self.data.groupBy("s").pivot("sample_id").max("v")
        
    def col_rename(self):
        new_columns = [col.replace(".", "") for col in self.data.columns]
        self.data = self.data.toDF(*new_columns)

In [8]:
# transform data
data_tr = DataTranspose(df)
data_tr.unpivot_data()
data_tr.pivot_data()
data_tr.col_rename()

In [9]:
# get transformed dataset
df = data_tr.data

In [10]:
# save the dataset to CSV
save_dataset(df, os.path.join(WORK_DIRECTORY, DATA_FILE_NAME_NEW))

23/02/18 17:04:54 WARN DAGScheduler: Broadcasting large task binary with size 22.0 MiB


23/02/18 17:11:47 WARN DAGScheduler: Broadcasting large task binary with size 81.5 MiB


23/02/18 17:18:47 WARN DAGScheduler: Broadcasting large task binary with size 6.1 MiB
